# Data analysis and visualization

### 1. Environment setup - initialization of Spark Session, installing libraries, and modules

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('DataFrame') \
    .master('local[*]') \
    .getOrCreate()

In [ ]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

In [ ]:
sc.install_pypi_package("pandas==1.0.5")
sc.install_pypi_package("matplotlib")

In [ ]:
# list installed packages

sc.list_packages()

In [ ]:
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.sql import Window
from pyspark.sql.functions import year
import matplotlib.pyplot as plt
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import concat_ws

### 2. Read Parquet files into DataFrames

In [ ]:
Badges_DF = spark.read.parquet("s3://bigdata-parquet-data/Badges.parquet/")

In [ ]:
Comments_DF = spark.read.parquet("s3://bigdata-parquet-data/Comments.parquet/")

In [ ]:
PostHistory_DF = spark.read.parquet("s3://bigdata-parquet-data/PostHistory.parquet/")

In [ ]:
PostLinks_DF = spark.read.parquet("s3://bigdata-parquet-data/PostLinks.parquet/")

In [ ]:
Posts_DF = spark.read.parquet("s3://bigdata-parquet-data/Posts.parquet/")

In [ ]:
Tags_DF = spark.read.parquet("s3://bigdata-parquet-data/Tags.parquet/")

In [ ]:
Users_DF = spark.read.parquet("s3://bigdata-parquet-data/Users.parquet/")

In [ ]:
Votes_DF = spark.read.parquet("s3://bigdata-parquet-data/Votes.parquet/")

### 3. Analysis

### a) number of posts over time

In [ ]:
# check first post creation date

Posts_DF.select("CreationDate").orderBy("CreationDate").show(1,truncate=False)

In [ ]:
# check last post creation date

Posts_DF.select("CreationDate").orderBy(Posts_DF.CreationDate.desc()).show(1,truncate=False)

In [ ]:
# add new column 'CreationYear' do Posts_DF DataFrame

Posts_DF2 = Posts_DF.withColumn("CreationYear",year(Posts_DF.CreationDate))
Posts_DF2.show(2,vertical=True,truncate=False)

In [ ]:
# create lineplot no.1

posts_count_over_time = Posts_DF2.groupBy("CreationYear").count().orderBy("CreationYear").toPandas()

plt.clf()
posts_count_over_time.plot(kind="area",x="CreationYear",y="count", rot=70, color="c", legend=None, figsize=(6,4))
plt.xticks(posts_count_over_time.CreationYear)
plt.xlim(2016, 2023)
plt.title("Number of posts across time")
plt.xlabel("CreationYear")
plt.ylabel("Number of posts")
plt.tight_layout()

%matplot plt

In [ ]:
# create lineplot no.2

posts_count_over_time = Posts_DF2.groupBy("CreationYear").count().orderBy("CreationYear").toPandas()

plt.figure(figsize=(6,4))
plt.plot(posts_count_over_time["CreationYear"], posts_count_over_time["count"], color="c", marker='o')
plt.xlabel("CreationYear")
plt.ylabel("Number of posts")
plt.title("Number of posts across time")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

%matplot plt

In [ ]:
# count posts in each year

Posts_DF2.groupBy("CreationYear").count().orderBy("CreationYear").show()

### b) forum time of 10 longest active users (from the user's creation to the last post/comment), excluding bots

#### - bots detection 

In [ ]:
# check AccountId's

Users_DF.select("AccountId").distinct().orderBy(f.asc("AccountId")).show()

In [ ]:
# based on aboves result, potentially AccountId== -1 is a bot, to confirm that check column "AboutMe" for those AccountId's

Users_DF.select("AboutMe").where(Users_DF.AccountId.like("-1")).show(4, vertical=True,truncate=False)

In [ ]:
# check if bots are describing themselves in the most likely way (by using word "bot")

Users_DF.select("AccountId","AboutMe").where(Users_DF.AboutMe.like("% I am bot %")).show(truncate=False) 

#### - count forum time 

In [ ]:
# prepare 3 DataFrames for join (rename column names and reduce columns in DataFrame) 

Users_DF_to_join = Users_DF.select("Id","CreationDate").withColumnRenamed("Id","User_Id")\
                   .withColumnRenamed("CreationDate","User_CreationDate")

Comments_DF_to_join = Comments_DF.select("Id","CreationDate","PostId","UserId")\
                                 .withColumnRenamed("Id","Comment_Id")\
                                 .withColumnRenamed("CreationDate","Comment_CreationDate")\
                                 .withColumnRenamed("UserId","Comment_UserId")

Posts_DF_to_join = Posts_DF.select("Id","CreationDate").withColumnRenamed("Id","Post_Id")\
                           .withColumnRenamed("CreationDate","Post_CreationDate")

In [ ]:
# join above DataFrames into one DataFrame

UCP_join_DF = Users_DF_to_join.join(Comments_DF_to_join, Users_DF_to_join.User_Id==Comments_DF_to_join.Comment_UserId)\
                              .join(Posts_DF_to_join, Comments_DF_to_join.PostId==Posts_DF_to_join.Post_Id)

In [ ]:
# checking on NULL values

UCP_join_DF.where(UCP_join_DF.User_CreationDate.isNull()).show()

In [ ]:
# extract difference in creation date between User_CreationDate and Comment_CreationDate/Post_CreationDate using Spark SQL

UCP_join_DF.createOrReplaceTempView("UCP_join_DF")

ForumTime_DF = spark.sql("SELECT User_Id, CASE WHEN\
DATEDIFF(DAY,User_CreationDate,Comment_CreationDate) > DATEDIFF(DAY,User_CreationDate,Post_CreationDate)\
THEN DATEDIFF(DAY,User_CreationDate,Comment_CreationDate) ELSE DATEDIFF(DAY,User_CreationDate,Post_CreationDate)\
END AS ForumTime FROM UCP_join_DF WHERE User_Id != '-1' ORDER BY ForumTime DESC")

In [ ]:
# add row number to ForumTime_DF using window function

windowSpec = Window.partitionBy("User_Id").orderBy(f.desc("ForumTime"))

ForumTime_DF2 = ForumTime_DF.withColumn("rownum", f.row_number()\
                                        .over(windowSpec)).orderBy(f.asc("rownum"),f.desc("ForumTime")).limit(10)

ForumTime_DF3 = ForumTime_DF2.select("User_Id","ForumTime").toPandas()

In [ ]:
# create barplot

forum_time = ForumTime_DF3["ForumTime"]


fig, ax = plt.subplots()
ax.bar(range(len(ForumTime_DF3.User_Id)), ForumTime_DF3.ForumTime, width=0.8, color='c')

for i, v in enumerate(forum_time):
    ax.text(i, v + 0.5, str(v), ha="center")

ax.set_title("TOP10 longest active users forum time")
ax.set_xlabel("User_Id")
ax.set_ylabel("ForumTime in days")
ax.set_xticks(range(len(ForumTime_DF3.User_Id)))
ax.set_xticklabels(ForumTime_DF3.User_Id)

    
%matplot plt

In [ ]:
# compare barplot with DataFrame

ForumTime_DF2.select("User_Id","ForumTime").show()

### c) comparison of the highest and lowest rated questions (length, tags, number of answers)

In [ ]:
# check max&min questions score (qestions: PostTypeId==1)

Posts_DF.where(Posts_DF.PostTypeId==1).select(f.max(Posts_DF.Score),f.min(Posts_DF.Score)).show()

In [ ]:
# collect records with max&min "Score" values

Posts_DF_max = Posts_DF.where(Posts_DF.PostTypeId==1).agg(f.max("Score")).collect()[0][0]
Posts_DF_min = Posts_DF.where(Posts_DF.PostTypeId==1).agg(f.min("Score")).collect()[0][0]

In [ ]:
Posts_DF_questions = Posts_DF.select("Id","PostTypeID","Body","AnswerCount","Tags","Score")\
                             .filter((f.col("Score") == Posts_DF_max) | (f.col("Score") == Posts_DF_min))

In [ ]:
# add new columns - question length and tags count

Posts_DF_all=Posts_DF_questions.withColumn("BodyLength",f.length(Posts_DF_questions.Body))\
                               .withColumn("TagsCount",f.size(f.split(f.col("Tags"), ",")).alias("TagCount"))

In [ ]:
Posts_DF_all.select("Id","PostTypeID","BodyLength","Score","Tags","TagsCount","AnswerCount").show(truncate=False) 

### d) percentage of cases where the highest-rated answer is unaccepted answer

In [ ]:
# create answers DatFrame (answers: PostTypeId==2)

Posts_DF_A=Posts_DF.select(f.col("Id").alias("AnswerId"),f.col("Score").alias("AnswerScore")\
                           ,f.col("ParentId").alias("QuestionId")).where(Posts_DF.PostTypeId==2)

In [ ]:
# create questions with answers DataFrame (qestions: PostTypeId==1)

Posts_DF_Q = Posts_DF.select(f.col("Id").alias("QuestionId"),f.col("Score").alias("QuestionScore")\
                             ,"AcceptedAnswerId").where(Posts_DF.PostTypeId==1)

In [ ]:
# join questions DF and answers DF

Posts_DF_QA = Posts_DF_Q.join(Posts_DF_A, Posts_DF_Q.QuestionId==Posts_DF_A.QuestionId)\
              .select(Posts_DF_Q.QuestionId,"AcceptedAnswerId","AnswerId","AnswerScore").orderBy(f.asc("QuestionId"))

Posts_DF_QA.show()

In [ ]:
# add row number to Posts_DF_QA

windowSpec = Window.partitionBy("QuestionId").orderBy(f.desc("AnswerScore"))
Posts_DF_QA_RN = Posts_DF_QA.withColumn("row_num",f.row_number().over(windowSpec))

In [ ]:
# count highest scored not accepted answers

Q_notAccepted_HS = Posts_DF_QA_RN.where((Posts_DF_QA_RN.AcceptedAnswerId.isNull()) & (Posts_DF_QA_RN.row_num==1)).count()

In [ ]:
# count all questions with answers

QA_all = Posts_DF.where((Posts_DF.AnswerCount > 0) & (Posts_DF.PostTypeId==1)).count()

In [ ]:
result=round((Q_notAccepted_HS/QA_all*100),2)
print(f"Percentage of cases where the highest-rated answer is unaccepted answer: {result}%")

### e) distribution of accepted answers vs. other answers (avg, deviation, min, max)

#### based on Posts_DF_QA DataFrame from point d

In [ ]:
# accepted answers

A_accepted = Posts_DF_QA.where(Posts_DF_QA.AcceptedAnswerId.isNotNull()).\
                  select(f.max(Posts_DF_QA.AnswerScore).alias("max_score"),\
                  f.min(Posts_DF_QA.AnswerScore).alias("min_score"),\
                  f.avg(Posts_DF_QA.AnswerScore).alias("avg_score"),\
                  f.stddev(Posts_DF_QA.AnswerScore).alias("std_score"))\
                  .withColumn("answers type", f.lit("accepted"))

A_accepted2 = A_accepted.select("answers type","max_score","min_score",f.round("avg_score",scale=2)\
                                .alias("avg_score"),f.round("std_score",scale=2).alias("std_score"))

A_accepted2.show()

In [ ]:
# not accepted answers

A_not_accepted = Posts_DF_QA.where(Posts_DF_QA.AcceptedAnswerId.isNull()).\
                  select(f.max(Posts_DF_QA.AnswerScore).alias("max_score"),\
                  f.min(Posts_DF_QA.AnswerScore).alias("min_score"),\
                  f.avg(Posts_DF_QA.AnswerScore).alias("avg_score"),\
                  f.stddev(Posts_DF_QA.AnswerScore).alias("std_score"))\
                  .withColumn("answers type", f.lit("not accepted"))

A_not_accepted2 = A_not_accepted.select("answers type","max_score","min_score",f.round("avg_score",scale=2)\
                                        .alias("avg_score"),f.round("std_score",scale=2).alias("std_score"))

A_not_accepted2.show()

In [ ]:
# dsiplay above DataFrames together

DF_final = A_accepted2.union(A_not_accepted2)
DF_final.show()

### f) top N tags that generated the most views

In [ ]:
# get TOP 10 tags with most views - first method

Tags_view = Posts_DF.select("Tags","ViewCount").orderBy(f.desc("ViewCount")).limit(10)
Tags_view.show(truncate=False)

In [ ]:
# get TOP 10 tags with most views - second method using Spark SQL

Posts_DF.createOrReplaceTempView("Posts_DF")

spark.sql("SELECT Tags,ViewCount FROM Posts_DF ORDER BY ViewCount DESC").show(10, truncate=False)

### g) number of posts over time for each of the top N tags (lineplot/barplot)

In [ ]:
# check TOP 3 tags

Tags_DF_TOP3=Tags_DF.select("TagName","Count").orderBy(Tags_DF.Count.desc()).limit(3)
Tags_DF_TOP3.show()

In [ ]:
# add "CreationYear" column to Posts_DF
 
Posts_DF_year = Posts_DF.withColumn("CreationYear",year(Posts_DF.CreationDate))
Posts_DF_year.show(2,vertical=True,truncate=False)

In [ ]:
# reduce number of columns (can be skipped)

Posts_DF_year2=Posts_DF_year.select("Tags","CreationYear")
Posts_DF_year2.show(truncate=False)

In [ ]:
# create DataFrames for each of TOP 3 tags

posts_count_over_time_top1 = Posts_DF_year.where(Posts_DF_year.Tags.like("%creality-ender-3%"))\
                                           .groupBy("CreationYear").count().orderBy("CreationYear").toPandas()

posts_count_over_time_top2 = Posts_DF_year.where(Posts_DF_year.Tags.like("%marlin%"))\
                                           .groupBy("CreationYear").count().orderBy("CreationYear").toPandas()

posts_count_over_time_top3 = Posts_DF_year.where(Posts_DF_year.Tags.like("%print-quality%"))\
                                           .groupBy("CreationYear").count().orderBy("CreationYear").toPandas()

In [ ]:
# lineplot

plt.figure(figsize=(6,4))
plt.plot(posts_count_over_time_top1["CreationYear"], posts_count_over_time_top_1["count"], marker='o', label="creality-ender-3")
plt.plot(posts_count_over_time_top2["CreationYear"], posts_count_over_time_top_2["count"], marker='o', label="marlin")
plt.plot(posts_count_over_time_top3["CreationYear"], posts_count_over_time_top_3["count"], marker='o', label="print-quality")
plt.xlabel("Year")
plt.ylabel("Posts count")
plt.title("Number of posts over years for TOP 3 tags")
plt.xticks(rotation=45)
plt.tight_layout()
plt.legend()
plt.show()
%matplot plt

### h) most frequently appearing words in titles (excluding stopwords)

In [ ]:
# clean "Title" column, transform string into array

Posts_DF_array = Posts_DF.withColumn("Title",f.regexp_replace("Title","\?",""))\
                           .withColumn("Title",f.regexp_replace("Title","\"",""))\
                           .select(f.split(f.col("Title")," ").alias("TitleArray"))\
                           .drop("Title").dropna()


Posts_DF_array.printSchema()

Posts_DF_array.show(10,truncate=False)

In [ ]:
#stopwords remover

swr = StopWordsRemover(inputCol="TitleArray", outputCol="TitleCleaned")

Posts_DF_clean = swr.transform(Posts_DF_array)

In [ ]:
Posts_DF_clean.show()

In [ ]:
# add new string column "TitleCleaned" and remove column "TitleAarray"

Posts_DF_clean2 = Posts_DF_clean.withColumn("TitleCleaned",concat_ws(",",f.col("TitleCleaned"))).drop("TitleArray").dropna()
Posts_DF_clean2.printSchema()
Posts_DF_clean2.show(5,truncate=False)

In [ ]:
# explode and split "TitleCleaned" to count most-appearing words

Posts_DFF = Posts_DF_clean2.withColumn("MostAppearing", f.explode(f.split(f.col("TitleCleaned"), ",")))\
  .groupBy("MostAppearing")\
  .count()\
  .sort("count", ascending=False)


Posts_DFF.show(15)

In [ ]:
# exclude "-" and NULL values, reduce output to TOP 15 words

Posts_DFF2 = Posts_DFF.filter((Posts_DFF.MostAppearing != "")&(Posts_DFF.MostAppearing != "-")\
                              &(Posts_DFF.MostAppearing != "3")).limit(15)
Posts_DFF2.show()

### i) percentage of users who have never posted anything

In [ ]:
# register temp tables

PostHistory_DF.createOrReplaceTempView("PostHistory_DF")
Users_DF.createOrReplaceTempView("Users_DF")

In [ ]:
# count number of users without any posts

Users_without_posts=spark.sql("SELECT DISTINCT(Id) FROM Users_DF WHERE Id NOT IN\
(SELECT DISTINCT(UserId) FROM PostHistory_DF)").count()

In [ ]:
# count all users

Users_all=Users_DF.select("Id").distinct().count()

In [ ]:
result=round((Users_without_posts/Users_all*100),2)
print(f"Percentage of users who have never posted anything: {result}%")

### j) average time from the appearance of the question to the appearance of the accepted answer

In [ ]:
# posts with accepted answers

Questions_DF = Posts_DF.select(f.col("Id").alias("Post_Id"),f.col("CreationDate")\
                               .alias("Post_CreationDate"),"AcceptedAnswerId")\
                               .where((Posts_DF.PostTypeId==1)&(Posts_DF.AcceptedAnswerId.isNotNull()))

Questions_DF.show(truncate=False)

In [ ]:
# all answers

Answers_DF = Posts_DF.select(f.col("Id").alias("Answer_Id"),f.col("CreationDate")\
                             .alias("Answer_CreationDate"),f.col("ParentId").alias("Question_Id")).where(Posts_DF.PostTypeId==2)
Answers_DF.show(truncate=False)

In [ ]:
# left join questions DF with answers DFto get only accepted answers

QA_DF = Questions_DF.join(Answers_DF, Questions_DF.AcceptedAnswerId == Answers_DF.Answer_Id,how = "left")
QA_DF.orderBy(f.asc("Post_Id")).show(truncate=False)

In [ ]:
# exclude NULL values

QA_DF2 = QA_DF.where(QA_DF.Answer_Id.isNotNull())

In [ ]:
# count difference between Post_CreationDate and Answer_CreationDate

QA_DF3 = QA_DF2.withColumn("difference",(f.unix_timestamp(QA_DF2.Answer_CreationDate)-\
                                         f.unix_timestamp(QA_DF2.Post_CreationDate)))

QA_DF3.orderBy(f.desc("difference")).show(vertical=True,truncate=False)

In [ ]:
# count average difference 

QA_avg_dif = QA_DF3.select(f.round((f.avg("difference")/86400),scale=2).alias("Average Accepted Answer Creation Time [days]"))
QA_avg_dif.show()